<br> Radley Ciego </br>
<br> GTECH 73100 </br>
<br> Assignment 8: Pandas & Geopandas </br>

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
data = "https://raw.githubusercontent.com/TangoYankee/gtech_731-geocomp-hw/main/assignment-eight/data/gz_2010_us_050_00_20m.geojson"
counties = gpd.read_file(data)
counties.head(10)

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01001,01,001,Autauga,County,594.436,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,0500000US01009,01,009,Blount,County,644.776,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
2,0500000US01017,01,017,Chambers,County,596.531,"POLYGON ((-85.18413 32.87053, -85.12342 32.772..."
3,0500000US01021,01,021,Chilton,County,692.854,"POLYGON ((-86.51734 33.02057, -86.51596 32.929..."
4,0500000US01033,01,033,Colbert,County,592.619,"POLYGON ((-88.13999 34.58170, -88.13925 34.587..."
5,0500000US01045,01,045,Dale,County,561.150,"POLYGON ((-85.41644 31.61947, -85.41743 31.314..."
6,0500000US01051,01,051,Elmore,County,618.485,"POLYGON ((-86.33678 32.76747, -86.31948 32.753..."
7,0500000US01065,01,065,Hale,County,643.943,"POLYGON ((-87.42120 32.87451, -87.47217 32.830..."
8,0500000US01079,01,079,Lawrence,County,690.678,"POLYGON ((-87.10507 34.68604, -87.10591 34.587..."
9,0500000US01083,01,083,Limestone,County,559.936,"POLYGON ((-86.83631 34.99176, -86.82066 34.991..."


In [13]:
data = "/Users/radleyciego/GitHub/webGIS/labs/data/map-of-europe/GeoJSON/europe.geojson"
europe = gpd.read_file(data)
europe.head(10)

,FID,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,LAT,geometry
0,0.0,AJ,AZ,AZE,31,Azerbaijan,8260,8352021,142,145,47.395,40.430,"MULTIPOLYGON (((45.08332 39.76804, 45.26639 39..."
1,0.0,AL,AL,ALB,8,Albania,2740,3153731,150,39,20.068,41.143,"POLYGON ((19.43621 41.02107, 19.45055 41.06000..."
2,0.0,AM,AM,ARM,51,Armenia,2820,3017661,142,145,44.563,40.534,"MULTIPOLYGON (((45.57305 40.63249, 45.52888 40..."
3,0.0,BK,BA,BIH,70,Bosnia and Herzegovina,5120,3915238,150,39,17.786,44.169,"POLYGON ((17.64984 42.88908, 17.57853 42.94382..."
4,0.0,BU,BG,BGR,100,Bulgaria,11063,7744591,150,151,25.231,42.761,"POLYGON ((27.87917 42.84110, 27.89500 42.80250..."
5,0.0,CY,CY,CYP,196,Cyprus,924,836321,142,145,33.219,35.043,"POLYGON ((33.65262 35.35410, 33.71305 35.38194..."
6,0.0,DA,DK,DNK,208,Denmark,4243,5416945,150,154,9.264,56.058,"MULTIPOLYGON (((11.51389 54.82972, 11.56444 54..."
7,0.0,EI,IE,IRL,372,Ireland,6889,4143294,150,154,-8.152,53.177,"MULTIPOLYGON (((-9.65639 53.22222, -9.66333 53..."
8,0.0,EN,EE,EST,233,Estonia,4239,1344312,150,154,25.793,58.674,"MULTIPOLYGON (((23.99083 58.10000, 23.97805 58..."
9,0.0,AU,AT,AUT,40,Austria,8245,8291979,150,155,14.912,47.683,"POLYGON ((13.83361 48.77361, 13.85806 48.77055..."


In [9]:
states = pd.read_json('./stateCodeToFips.json', orient='index') 
states.head(5)

,0
AK,2
AL,1
AR,5
AS,60
AZ,4


In [12]:
states.columns=['STATE'] # rename fips code column to STATE
states.dtypes['STATE'] # check datatype of STATE column for join

dtype('int64')

## Task 1:
Find top n most common county names (n = 3 or 5, for example)

In [ ]:
def top_5_counties(n):
    return counties.groupby("NAME").size().nlargest(n)

In [ ]:
top_5_counties(5)

## Task 2:
Derive summary statistics for states, for example, number of counties, counties with max/min area

In [ ]:
lrg_cnties = counties.groupby("NAME").max("CENSUSAREA")
lrg_cnties.tail(5)

In [ ]:
smll_cnties = counties.groupby("NAME").min("CENSUSAREA")
smll_cnties.head(5)

## Task 3:
Join fips code to get full names of states (plesae do a real join using merge)

## Task 4:
Plot maps to show all the counties with the most common names from first task